In [18]:
# Bot to examine Commons category and add SDC statements for Met object ID:
#   Met Object ID (P3634) -> number
#   collection (P195) -> Q160236

import pywikibot
from pywikibot import pagegenerators
from pywikibot.comms import http

import json
import re
from itertools import islice

# Parameters specific to the institution
PARAMS = {
    'test_mode': True,
    'category': u'Category:Department of Ancient Near Eastern Art, Metropolitan Museum of Art',
    'recurse': True,
    'search_string': r'source[ ]*=[ ]*http[s]*://www.metmuseum.org/art/collection/search/(\d+)',
    'pid':'P3634',
    'pid_summary': u'importing Met Object ID from source info from Commons template',
    'institution_pid':'P195',
    'institution_qid':'Q160236',
    'institution_summary': u'adding collection based on Met Object ID'
}

counter = {
    'id': 0,
    'institution': 0,
    'id_skip': 0,
    'institution_skip': 0,
    'unmatched': 0
}

def addClaim(mediaid, pid, qid, summary='') -> int:
    """
    :param mediaid:
    :param pid:
    :param qid:
    :param summary:
    :return:
    """
    pywikibot.output(u'Adding %s->%s to %s. %s' % (pid, qid, mediaid, summary))

    # Check for existing entry - if it exists at all, skip and honor existing entry
    request = site._simple_request(action='wbgetentities',ids=mediaid)
    data = request.submit()
    if (data.get(u'entities').get(mediaid).get(u'statements').get(pid)):
        pywikibot.output(u'  Existing entry: skipping to be safe.')
        return 0  # Skip

    tokenrequest = http.fetch(u'https://commons.wikimedia.org/w/api.php?action=query&meta=tokens&type=csrf&format=json')

    tokendata = json.loads(tokenrequest.text)
    token = tokendata.get(u'query').get(u'tokens').get(u'csrftoken')

    postvalue = {"entity-type":"item","numeric-id": qid.replace(u'Q', u'')}

    postdata = {u'action' : u'wbcreateclaim',
                u'format' : u'json',
                u'entity' : mediaid,
                u'property' : pid,
                u'snaktype' : u'value',
                u'value' : json.dumps(postvalue),
                u'token' : token,
                u'summary' : summary
                }

    if PARAMS['test_mode']:
        # print ('TEST_MODE: addClaim:', postdata)
        pass
    else:
        apipage = http.fetch(u'https://commons.wikimedia.org/w/api.php', method='POST', data=postdata)

    return 1  # Successful add

def addClaimString(mediaid, pid, instring, summary=''):
    """
    :param mediaid:
    :param pid:
    :param qid:
    :param summary:
    :return:
    """
    pywikibot.output(u'Adding %s->%s to %s. %s' % (pid, instring, mediaid, summary))

    # Check for existing entry - if it exists at all, skip and honor existing entry
    request = site._simple_request(action='wbgetentities',ids=mediaid)
    data = request.submit()
    if (data.get(u'entities').get(mediaid).get(u'statements').get(pid)):
        pywikibot.output(u'  Existing entry: skipping to be safe.')
        return 0
    
    tokenrequest = http.fetch(u'https://commons.wikimedia.org/w/api.php?action=query&meta=tokens&type=csrf&format=json')

    tokendata = json.loads(tokenrequest.text)
    token = tokendata.get(u'query').get(u'tokens').get(u'csrftoken')

    postdata = {u'action' : u'wbcreateclaim',
                u'format' : u'json',
                u'entity' : mediaid,
                u'property' : pid,
                u'snaktype' : u'value',
                u'value' : '"'+instring+'"',
                u'token' : token,
                u'summary' : summary
                }

    if PARAMS['test_mode']:
        # print ('TEST_MODE: addClaim:', postdata)
        pass
    else:
        apipage = http.fetch(u'https://commons.wikimedia.org/w/api.php', method='POST', data=postdata)

    return 1    


def media_id(page: pywikibot.page.FilePage) -> str:
    """Return MID from page ID: just prepend an M"""
    return u'M%s' % (page.pageid,)

site = pywikibot.Site(u'commons', u'commons')
cat = pywikibot.Category(site,PARAMS['category'])
gen = pagegenerators.CategorizedPageGenerator(cat, recurse=PARAMS['recurse'])

patt = re.compile(PARAMS['search_string'])

# for page in gen:
for page in islice(gen, 100):
    #Do something with the page object, for example:
    text = page.text
    mediaid = media_id(page)

    m = patt.search(text)
    if m:
        id_string = m.group(1)
        # print ('  ', id_string)
        summary = PARAMS['pid_summary']
        result = addClaimString(mediaid, PARAMS['pid'], id_string, summary)
        if result>0:
            counter['id'] += 1
        else:
            counter['id_skip'] += 1

        if 'institution_pid' in PARAMS:
            institution = PARAMS['institution_qid']  # Met
            summary = PARAMS['institution_summary']
            result = addClaim(mediaid, PARAMS['institution_pid'], institution, summary)
            if result>0:
                counter['institution'] += 1
            else:
                counter['institution_skip'] += 1
    else:
        counter['unmatched'] += 1
        pywikibot.output(u'Unmatched %s' % (mediaid, ))

    print ()

print ('Final report')
{print ('%s: %s' % (x, counter[x])) for x in counter}

Adding P3634->325930 to M60514717. importing Met Object ID from source info from Commons template
Adding P195->Q160236 to M60514717. adding collection based on Met Object ID


Adding P3634->325443 to M60518801. importing Met Object ID from source info from Commons template
Adding P195->Q160236 to M60518801. adding collection based on Met Object ID


Adding P3634->326168 to M60516990. importing Met Object ID from source info from Commons template
Adding P195->Q160236 to M60516990. adding collection based on Met Object ID


Adding P3634->324466 to M60509643. importing Met Object ID from source info from Commons template
Adding P195->Q160236 to M60509643. adding collection based on Met Object ID


Adding P3634->324474 to M60509662. importing Met Object ID from source info from Commons template
Adding P195->Q160236 to M60509662. adding collection based on Met Object ID


...

Adding P3634->326484 to M60518673. importing Met Object ID from source info from Commons template
Adding P195->Q160236 to M60518673. adding collection based on Met Object ID


Adding P3634->326487 to M60518677. importing Met Object ID from source info from Commons template
Adding P195->Q160236 to M60518677. adding collection based on Met Object ID


Adding P3634->323100 to M60515924. importing Met Object ID from source info from Commons template
Adding P195->Q160236 to M60515924. adding collection based on Met Object ID



Final report
id: 99
institution: 99
id_skip: 0
institution_skip: 0
unmatched: 1


{None}